In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import hexapod_plot_utils as hpu # plots nice stuff
plt.rcParams["figure.figsize"] = (10,5)

# Define leg parameters
coxa_len = 28.75
tibia_len = 68.825
femur_len = 40
z_offset = tibia_len

# Define body parameters (for rotational IK)
X0_LEN = 45.768
Y0_LEN = 26.424
Y1_LEN = 52.848
R1_ORIGIN = np.array( (X0_LEN, Y0_LEN, z_offset) )
R2_ORIGIN = np.array( (0.0, Y1_LEN, z_offset) )
R3_ORIGIN = np.array( (-X0_LEN, Y0_LEN, z_offset) )
L1_ORIGIN = np.array( (X0_LEN, -Y0_LEN, z_offset) )
L2_ORIGIN = np.array( (0.0, -Y1_LEN, z_offset) )
L3_ORIGIN = np.array( (-X0_LEN, -Y0_LEN, z_offset) )
leg_origins = np.array( (R1_ORIGIN, R2_ORIGIN, R3_ORIGIN, L3_ORIGIN, L2_ORIGIN, L1_ORIGIN) )
print(leg_origins.shape)

def to_rads(num):
    return num*np.pi/180

def to_degs(num):
    return num/np.pi*180

def pythagoras(nums):
    return np.sqrt(np.sum(np.power(nums, 2), axis=1))

(6, 3)


In [2]:
# Swing: Leg origin does not move; leg tip moves
def swing(tip_coords):
    coxa_angle_ik = np.arctan2(tip_coords[0], tip_coords[1]) # 'gamma' in notes

    coxa_len_ik = coxa_len * np.cos( coxa_angle_ik ) # Takes side view of this thing.
    femur_len_ik = femur_len * np.cos( coxa_angle_ik )
    tibia_len_ik = tibia_len * np.cos( coxa_angle_ik )

    y_offset_ik = tip_coords[1] - coxa_len_ik
    z_offset_ik = z_offset - tip_coords[2]
    l_len = np.sqrt( z_offset_ik**2 + y_offset_ik**2 )

    print("z_offset_ik: %0.4f | y_offset_ik: %0.4f | l_len: %0.4f" %(z_offset_ik, y_offset_ik, l_len) )

    femur_angle_ik1 = np.arccos( z_offset_ik / l_len )
    # print(z_offset_ik / l_len)
    femur_angle_ik2 = np.arccos( (femur_len_ik**2 + l_len**2 - tibia_len_ik**2) / (2*femur_len_ik*l_len) )
    # print((femur_len**2 + l_len**2 - tibia_len**2) / (2*femur_len*l_len))
    # print(to_degs(femur_angle_ik1), to_degs(femur_angle_ik2))
    femur_angle_ik = femur_angle_ik1 + femur_angle_ik2

    tibia_angle_ik = np.arccos( (femur_len_ik**2 - l_len**2 + tibia_len_ik**2) / (2*femur_len_ik*tibia_len_ik) )
    # print((femur_len**2 - l_len**2 + tibia_len**2) / (2*femur_len*tibia_len))

    angles_ik = [to_degs(coxa_angle_ik)+90, to_degs(femur_angle_ik), to_degs(tibia_angle_ik)]

    return angles_ik

In [3]:
# Stance: Leg tip does not move, leg origin moves.
# Modify the swing IK to frame leg tip in terms of origin
# Origin is at (0,0,68.825)
def stance(origin_coords):
    pass

In [4]:
# Body IK
# Return a np array of coordinates. The arguments of these coordinates are relative to a neutral position.
def body_roll(origin_coords, roll):
    new_coords = np.copy( origin_coords )
    new_coords[:, 1] = origin_coords[:, 1] * np.cos( to_rads(roll) ) # Y
    new_coords[:, 2] += origin_coords[:, 1] * np.sin( to_rads(roll) ) # Z

    # print("origin_coords:\n", origin_coords)
    # print("new_coords:\n", new_coords)
    
    return new_coords

def body_pitch(origin_coords, pitch):
    new_coords = np.copy( origin_coords )
    new_coords[:, 0] = origin_coords[:, 0] * np.cos( to_rads(pitch) ) # X
    new_coords[:, 2] -= origin_coords[:, 0] * np.sin( to_rads(pitch) ) # Z (L2 and R2 are unchanged)
    new_coords[1, 2] = origin_coords[1, 2]
    new_coords[4, 2] = origin_coords[4, 2]

    # print("origin_coords:\n", origin_coords)
    # print("new_coords:\n", new_coords)

    return new_coords

def body_rotate(origin_coords, theta):
    new_coords = np.copy( origin_coords )
    distances = np.empty(0)
    angles = np.empty(0)
    for x in range(origin_coords.shape[0]):
        distances = np.append(distances, pythagoras( origin_coords[x, 0:2] ) )
        angles = np.append(angles, to_degs(np.arctan2( origin_coords[x, 1], origin_coords[x, 0]))+theta )
    # print("distances:\n", distances)
    # print("angles:\n", angles)

    new_coords[:, 0] = distances * np.cos( to_rads( angles ) )
    new_coords[:, 1] = distances * np.sin( to_rads( angles ) )

    # print("origin_coords:\n", origin_coords)
    # print("new_coords:\n", new_coords)

    return new_coords

def body_translate_x(origin_coords, x):
    new_coords = np.copy( origin_coords )
    new_coords[:, 0] += x
    # print("origin_coords:\n", origin_coords)
    # print("new_coords:\n", new_coords)

    return new_coords

def body_translate_y(origin_coords, y):
    new_coords = np.copy( origin_coords )
    new_coords[:, 1] += y
    # print("origin_coords:\n", origin_coords)
    # print("new_coords:\n", new_coords)
    
    return new_coords

def body_translate_z(origin_coords, z):
    new_coords = np.copy( origin_coords )
    new_coords[:, 2] += z
    # print("origin_coords:\n", origin_coords)
    # print("new_coords:\n", new_coords)
    
    return new_coords

In [5]:
import numpy as np

# Robot coordinate class
class hexapod:
    # As always, leg order is (R123,L321) and coords are (x,y,z)
    # DO NOT WRITE TO THESE DIRECLTY - getter/setter
    _leg_end_loc = np.zeros((6,3)) # Local coordinates of leg tips
    _leg_ori_loc = np.zeros((6,3)) # local coordinates of leg origins (local to each leg)
    _leg_end_abs = np.zeros((6,3)) # absolute coordinates
    _leg_ori_abs = np.zeros((6,3))
    _leg_angle = np.zeros(6) # Each leg's coxa angle (gamma)
    _body_z = 68.825 # Body origin (0,0,z)
    _roll = 0.0
    _pitch = 0.0

    # body parameters
    X0_LEN = 45.768
    Y0_LEN = 26.424
    Y1_LEN = 52.848
    OFFSET_ROLL = np.array((Y0_LEN, Y1_LEN, Y0_LEN, -Y0_LEN, -Y1_LEN, -Y0_LEN)) # Roll modifies Z based on y-coord
    OFFSET_PITCH = np.array((X0_LEN, 0.0, -X0_LEN, -X0_LEN, 0.0, X0_LEN)) # Pitch modifies Z based on x-coord
    OFFSET_ANGLE = np.array((30, 90, 150, 210, 270, 330), dtype='float32') # Modify angle offset for each
    
    def __init__(self, leg_end_loc=None, leg_ori_loc=None, leg_end_abs=None, 
                leg_angle=np.array((90.0, 90.0, 90.0, 90.0, 90.0, 90.0)), body_z=68.825, roll=0.0, pitch=0.0):

        self._leg_angle = leg_angle # initialise these constants
        self._body_z = body_z
        self._roll = roll
        self._pitch = pitch

        # Set leg origins based on roll and pitch settings
        self._leg_ori_loc[:, 2] = self._body_z + self.OFFSET_ROLL*np.sin(self.to_rads(self._roll)) + self.OFFSET_PITCH*np.sin(self.to_rads(self._pitch))
        self._leg_ori_abs[:, 0] = self.OFFSET_PITCH*np.cos(self.to_rads(self._pitch))
        self._leg_ori_abs[:, 1] = self.OFFSET_ROLL*np.cos(self.to_rads(self._roll))

        if leg_end_loc is None: 
            if leg_end_abs is None:
                print("Specify local or absolute coordinates for leg tips!")
            else:
                print("Setting leg tips absolutely")
                self._leg_end_abs = leg_end_abs
                self._leg_end_loc[:, 2] = self._leg_end_abs[:, 2] # Z is absolute
                delta_xy = self._leg_end_abs[:, 0:2] - self._leg_ori_abs[:, 0:2]
                leg_len = self.pythagoras(delta_xy)
                # This way overwrites leg angle!
                self._leg_angle = self.to_degs( np.arctan2(delta_xy[1], delta_xy[0]) ) - self.OFFSET_ANGLE
                self._leg_end_loc[:, 0] = leg_len[:, 0] * np.sin(self.to_rads(self._leg_angle))
                self._leg_end_loc[:, 1] = leg_len[:, 1] * np.cos(self.to_rads(self._leg_angle))

        elif leg_end_abs is not None:
            print("Only local or absolute coordinates for leg tips should be given!")
        
        else:
            print("Setting leg tips locally")
            self._leg_end_loc = leg_end_loc
            self._leg_end_abs[:, 2] = self._leg_end_loc[:, 2] # Z is absolute
            leg_len = self.pythagoras(self._leg_end_loc[:, 0:2])
            phi = 180-self.OFFSET_ANGLE-self._leg_angle
            self._leg_end_abs[:, 0] = self._leg_ori_abs[:, 0] + leg_len*np.sin(self.to_rads(phi))
            self._leg_end_abs[:, 1] = self._leg_ori_abs[:, 1] + leg_len*np.cos(self.to_rads(phi))

    # Helper functions
    def to_rads(self, num):
        return num*np.pi/180

    def to_degs(self, num):
        return num/np.pi*180

    def pythagoras(self, nums):
        return np.sqrt(np.sum(np.power(nums, 2), axis=1))

    # Sets local value for leg tip, and changes other relevant values
    def set_leg_end_loc(self, newval):
        self._leg_end_loc = newval
        # TODO convert to end_abs

    # Sets absolute value for leg tip, and changes other relevant values
    def set_leg_end_abs(self, newval):
        self._leg_end_abs = newval
        # TODO convert to end_loc

    # Sets local value for leg origin, and changes other relevant values
    def set_leg_ori_loc(self, newval):
        self._leg_ori_loc = newval
        # TODO convert to ori_abs

    # Sets absolute value for leg tip, and changes other relevant values
    def set_leg_ori_abs(self, newval):
        self._leg_ori_abs = newval
        # TODO convert to ori_loc

    # Sets value for leg angle
    def set_leg_angle(self, newval):
        self._leg_angle = newval

    # Set new value for body origin
    def set_body_origin(self, newval):
        self._body_origin = newval
        # TODO Updates absolute position of _everything_

    def get_leg_end_loc(self):
        return self._leg_end_loc
    def get_leg_end_abs(self):
        return self._leg_end_abs
    def get_leg_ori_loc(self):
        return self._leg_ori_loc
    def get_leg_ori_abs(self):
        return self._leg_ori_abs
    def get_leg_angle(self):
        return self._leg_angle
    def get_body_origin(self):
        return self._body_origin
    def print_state(self): # Sanity check
        np.set_printoptions(precision=3, suppress=True) # print prettier
        print("Local leg endpoints:\n", self._leg_end_loc)
        print("Abs leg endpoints:\n", self._leg_end_abs)
        print("Local leg origin:\n", self._leg_ori_loc)
        print("Abs leg origin:\n", self._leg_ori_abs)
        print("Leg Angles:\n", self._leg_angle)
        print("Body roll:\n", self._roll)
        print("Body pitch:\n", self._pitch)

In [6]:
# Test creation of class
leg_end_coords = np.tile((0.0, 68.75, 0.0), (6,1)) # check back
# print(leg_end_coords, leg_end_coords.shape)
my_hex = hexapod(leg_end_loc=leg_end_coords)
my_hex.print_state()

hpu.plot_everything(my_hex.get_leg_end_abs(), my_hex.get_leg_ori_abs())

Setting leg tips locally
Local leg endpoints:
 [[ 0.   68.75  0.  ]
 [ 0.   68.75  0.  ]
 [ 0.   68.75  0.  ]
 [ 0.   68.75  0.  ]
 [ 0.   68.75  0.  ]
 [ 0.   68.75  0.  ]]
Abs leg endpoints:
 [[ 105.307   60.799    0.   ]
 [   0.     121.598    0.   ]
 [-105.307   60.799    0.   ]
 [-105.307  -60.799    0.   ]
 [  -0.    -121.598    0.   ]
 [ 105.307  -60.799    0.   ]]
Local leg origin:
 [[ 0.     0.    68.825]
 [ 0.     0.    68.825]
 [ 0.     0.    68.825]
 [ 0.     0.    68.825]
 [ 0.     0.    68.825]
 [ 0.     0.    68.825]]
Abs leg origin:
 [[ 45.768  26.424   0.   ]
 [  0.     52.848   0.   ]
 [-45.768  26.424   0.   ]
 [-45.768 -26.424   0.   ]
 [  0.    -52.848   0.   ]
 [ 45.768 -26.424   0.   ]]
Leg Angles:
 [90. 90. 90. 90. 90. 90.]
Body roll:
 0.0
Body pitch:
 0.0


In [7]:
leg_end_coords = [0.0, 68.75, 0.0] # check back
_ = hpu.plot_legs(swing(leg_end_coords))

z_offset_ik: 68.8250 | y_offset_ik: 40.0000 | l_len: 79.6045

Desired angles: ['90.0000', '90.0000', '90.0000']
Coxa length: 28.75
Femur length: 40.0
Tibia length: 68.825
Leg_base: ['0.0000', '0.0000', '68.8250']
coxa_end: ['0.0000', '28.7500', '68.8250']
femur_end:['0.0000', '68.7500', '68.8250']
tibia_end: ['0.0000', '68.7500', '-0.0000']
theta: 0.0000
